In [ ]:
import pandas as pd
import pandas_ta as ta
import random



candlestick_frame = 12
pnl_threshold = 3


df = pd.read_csv('../../data/ETHUSDT-15m.csv')

try:
    df.ta.strategy("all")
except Exception as e:
    print(e)

    ts = list(df['time'])
    open = list(df['open'])
    high = list(df['high'])
    low = list(df['low'])
    close = list(df['close'])
    volume = list(df['volume'])
    tot = len(ts)
    long_runup_lst = []
    long_drawdown_lst = []
    short_runup_lst = []
    short_drawdown_lst = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            max_high = max(high[idx+1:idx+candlestick_frame])
            min_low = min(low[idx+1:idx+candlestick_frame])
            entry_price = open[idx+1]
            long_runup_lst.append(round((max_high*100/entry_price)-100, 6))
            long_drawdown_lst.append(round((min_low*100/entry_price)-100, 6))
            short_runup_lst.append(round((entry_price*100/min_low)-100, 6))
            short_drawdown_lst.append(round((entry_price*100/max_high)-100, 6))
        else:
            long_runup_lst.append(0)
            long_drawdown_lst.append(0)
            short_runup_lst.append(0)
            short_drawdown_lst.append(0)     


    long=[]
    short=[]
    dont_trade=[]
    signal = []

    for idx in range(tot):
        if (idx >= candlestick_frame) and (idx <= tot - candlestick_frame):
            if long_runup_lst[idx] >= pnl_threshold:
                signal.append('long')
            elif short_runup_lst[idx] >= pnl_threshold:
                signal.append('short')
            else:
                signal.append('dont_trade')
        else:
            signal.append('dont_trade')


    df['signal'] = signal

    long_indices = df[df['signal'].str.contains('long', case=False)].index
    short_indices = df[df['signal'].str.contains('short', case=False)].index
    dont_trade_indices = list(df[df['signal'].str.contains('dont_trade', case=False)].index)
    num_indices_to_pick  = len(dont_trade_indices) - min([len(long_indices), len(short_indices)])
    random_indices = random.sample(dont_trade_indices, num_indices_to_pick)
    df = df.drop(random_indices)

    df.reset_index(drop=True, inplace=True)

    # df = df.drop(columns = ['time', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    
    print('saving dataset.csv....')
    dataset_fn = f'../../data/dataset.csv'
    pd.DataFrame(df).to_csv(dataset_fn, index=False)


